## Preparations

### Imports

In [1]:
import os
import requests
import tiktoken

from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from openai import OpenAI
from tqdm.auto import tqdm
from typing import Any, Dict, List

### Getting FAQ document

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Preparing OpenAI client and RAG

In [ ]:
load_dotenv('../.env')

In [3]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_PROXY_URL'),
)

In [4]:
def llm(prompt: str) -> str:
    """
    Generates a response based on the input prompt using GPT-4O.

    This method sends a user message to the GPT-4O chat completions API,
    receives a response, and extracts the content of the first choice made by the AI.

    Parameters
    ----------
    prompt : str
        The user's message or prompt to which the AI will respond.

    Returns
    -------
    str
        The content of the AI's response as a string.
    """
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}],
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query: str) -> str:
    """
    Performs a research assistant task given a query.

    This method searches for relevant information using Elasticsearch,
    builds a prompt based on the search results, generates an answer using a language model,
    and returns the generated answer.

    Parameters
    ----------
    query : str
        The user's query or question to which the research assistant will respond.

    Returns
    -------
    str
        The generated answer as a string.
    """
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

### Prepare Elasticsearch client

In [6]:
es_client = Elasticsearch('http://localhost:9200')

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        },
    },
}

index_name = "course-homework"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## [Homework questions](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md)

### [Q1. Running Elastic](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q1-running-elastic)

In [10]:
print('Q1:', es_client.info()['version']['build_hash'])

Q1: 42f05b9372a9a4a470db3b52817899b99a76ee73


### [Q2. Indexing the data](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q2-indexing-the-data)

In [11]:
print('Q2: index')

Q2: index


### [Q3. Searching](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q3-searching)

In [12]:
query = 'How do I execute a command in a running docker container?'

In [13]:
def elastic_search(query: str) -> List[float]:
    """
    Searches Elasticsearch for documents matching a given query and returns their scores.

    This method constructs a search query targeting documents related to "data-engineering-zoomcamp",
    executes the search against Elasticsearch, and collects the scores of the top matches.

    Parameters
    ----------
    query : str
        The search query to be executed against Elasticsearch.

    Returns
    -------
    List[float]
        A list of scores corresponding to the relevance of the top search results.
    """
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields",
                    },
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    },
                },
            },
        },
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [14]:
print('Q3: Top score is', elastic_search(query)[0])

Q3: Top score is 75.54128


### [Q4. Filtering](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q4-filtering)

In [15]:
def elastic_search(query: str) -> List[Dict[str, Any]]:
    """
    Searches Elasticsearch for documents matching a given query and returns their contents.

    This method constructs a search query targeting documents related to "machine-learning-zoomcamp",
    executes the search against Elasticsearch, and collects the contents (_source) of the top matches.

    Parameters
    ----------
    query : str
        The search query to be executed against Elasticsearch.

    Returns
    -------
    List[Dict[str, Any]]
        A list of dictionaries representing the contents of the top search results.
    """
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields",
                    },
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    },
                },
            },
        },
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [16]:
search_results = elastic_search(query)
q4_result = search_results[2]
print('Q4:', q4_result['question'])

Q4: How do I copy files from a different folder into docker container’s working directory?


### [Q5. Building a prompt](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q5-building-a-prompt)

In [17]:
def build_prompt(query: str, search_results: List[Dict[str, str]]) -> str:
    """
    Constructs a prompt for a language model based on a query and search results.

    This method formats a query and search results into a structured prompt,
    designed to guide a language model in generating an answer based on the provided context.

    Parameters
    ----------
    query : str
        The question or query to be answered.
    search_results : List[Dict[str, str]]
        A list of dictionaries, each containing a question and its corresponding answer.

    Returns
    -------
    str
        A formatted prompt that combines the question with the context from the search results.
    """
    context_template = '''
Q: {question}
A: {text}
'''.strip()
    
    prompt_template = '''
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
'''.strip()

    context = ''
    for doc in search_results:
        context = context + context_template.format(
            question=doc['question'],
            text=doc['text']
        ) + '\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [18]:
prompt = build_prompt(query, search_results)

print('Q5:', len(prompt))

Q5: 1462


### [Q6. Tokens](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q6-tokens)

In [19]:
def token_counter(prompt: str) -> int:
    """
    Counts the number of tokens in a given prompt using the 'gpt-4o' model's encoding.

    This method encodes the input prompt using the tokenization scheme associated with the 'gpt-4o' model,
    and returns the total count of tokens in the encoded representation.

    Parameters
    ----------
    prompt : str
        The input text whose token count is to be determined.

    Returns
    -------
    int
        The total number of tokens in the encoded version of the input prompt.
    """
    encoding = tiktoken.encoding_for_model('gpt-4o')
    tokens = encoding.encode(prompt)

    return len(tokens)

In [20]:
print('Q6:', token_counter(prompt), 'tokens')

Q6: 322 tokens


### [Bonus: generating the answer](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#bonus-generating-the-answer-ungraded)

In [21]:
rag(query)

'To execute a command in a running Docker container, you can use the `docker exec` command. First, you need to identify the container ID of the running container by using the `docker ps` command. Once you have the container ID, you can execute a command inside that container. Here’s how you can do it:\n\n1. Find the container ID of the running container:\n\n```bash\ndocker ps\n```\n\n2. Execute the command inside the running container:\n\n```bash\ndocker exec -it <container-id> <command>\n```\n\nFor example, to start a bash session in the running container, you would use:\n\n```bash\ndocker exec -it <container-id> bash\n```'

### [Bonus: calculating the costs](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#bonus-calculating-the-costs-ungraded)

In [22]:
result = (150 * 5 + 250 * 15) / 1_000_000 * 1_000

print(f'Total cost: {result}$')

Total cost: 4.5$
